In [0]:
# 23	39927502
# 24	22723874
# 25	1542830
df_23 = spark.sql(f'''
    select date_ym
        , log_create_time
        , context_name
        , message_id
        , device_name
        , platform_code as X_device_platform
        , platform_version as X_device_product
        , device_id as mac_addr
        , name
        , type
        , spec
        , label
        , id
        -- , device_id as mac_addr
        -- , country_code
        -- , sales_model_code
        -- , dpv
    from   aic_data_ods.tlamp_view.view__eim__nl_eim_inputlist
    where  country_code = 'US'
    and  platform_version = 'webOSTV 23'
    and  date_ym >= '2023-01'
    and  date_ym < '2025-12'
''')

df_24 = spark.sql(f'''
    select date_ym
        , log_create_time
        , context_name
        , message_id
        , device_name
        , platform_code as X_device_platform
        , platform_version as X_device_product
        , device_id as mac_addr
        , name
        , type
        , spec
        , label
        , id
    from   aic_data_ods.tlamp_view.view__eim__nl_eim_inputlist
    where  country_code = 'US'
    and  platform_version = 'webOSTV 24'
    and  date_ym >= '2024-01'
    and  date_ym < '2025-12'
''')

df_25 = spark.sql(f'''
    select date_ym
        , log_create_time
        , context_name
        , message_id
        , device_name
        , platform_code as X_device_platform
        , platform_version as X_device_product
        , device_id as mac_addr
        , name
        , type
        , spec
        , label
        , id
    from   aic_data_ods.tlamp_view.view__eim__nl_eim_inputlist
    where  country_code = 'US'
    and  platform_version = 'webOSTV 25'
    and  date_ym >= '2025-01'
    and  date_ym < '2025-12'
''')


In [0]:
# s3 csv
df_23.drop("mac_addr").coalesce(1).write.format("csv").option("header", "true").option("compression", "bzip2").mode("overwrite").save(
    "s3://s3-lge-he-inbound-aic-dev/HEDS/1171/raw_bz2/webos23"
)
df_24.drop("mac_addr").coalesce(1).write.format("csv").option("header", "true").option("compression", "bzip2").mode("overwrite").save(
    "s3://s3-lge-he-inbound-aic-dev/HEDS/1171/raw_bz2/webos24"
)
df_25.drop("mac_addr").coalesce(1).write.format("csv").option("header", "true").option("compression", "bzip2").mode("overwrite").save(
    "s3://s3-lge-he-inbound-aic-dev/HEDS/1171/raw_bz2/webos25"
)

In [0]:
from pyspark.sql.functions import countDistinct, count

df_agg_23 = df_23.groupBy("X_device_product").agg(countDistinct("mac_addr").alias("count_ud"), count("mac_addr").alias("count"))
df_agg_24 = df_24.groupBy("X_device_product").agg(countDistinct("mac_addr").alias("count_ud"), count("mac_addr").alias("count"))
df_agg_25 = df_25.groupBy("X_device_product").agg(countDistinct("mac_addr").alias("count_ud"), count("mac_addr").alias("count"))

df_union = df_agg_23.union(df_agg_24)
df_union = df_union.union(df_agg_25)

display(df_union)
